# Installations
- EurostatApiClient
- Geopandas

In [1]:
!pip install eurostatapiclient
!pip install geopandas

# Imports

In [2]:
# Jupyter notebook
from IPython.display import Markdown as md
from IPython import display
from base64 import b64decode
from ipywidgets import interact

# Eurostat data
from eurostatapiclient import EurostatAPIClient

# Maps
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
import geopandas as gpd
from geopandas.tools import geocode

# Data Science
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Some Randomness, for Fun

In [3]:
import random
random.seed(0) # pick your seed

# Data

In [4]:
VERSION = 'v2.1'
FORMAT = 'json'
LANGUAGE = 'en'
client = EurostatAPIClient(VERSION, FORMAT, LANGUAGE)

In [5]:
%%html
<iframe src="https://ec.europa.eu/eurostat/databrowser/view/lan_lcv_ovw/default/table?lang=en" width="1000" height="600"></iframe>

In [6]:
landcover_types = {'LCA': 'Artificial land',
                   'LCB': 'Cropland',
                   'LCC': 'Woodland',
                   'LCD': 'Shrubland',
                   'LCE': 'Grassland',
                   'LCF': 'Bare land',
                   'LCG': 'Water',
                   'LCH': 'Wetland'}

In [7]:
par_df1 = {
    'landcover': ['LCA', 'LCB', 'LCC', 'LCD', 'LCE', 'LCF', 'LCG', 'LCH'],
    'unit': 'PC',
    'geoLevel': 'nuts2',
}

df1 = client.get_dataset('lan_lcv_ovw', params=par_df1).to_dataframe()

df1.rename(columns={'time': 'year'}, inplace=True)
df1.drop(['unit'], axis=1, inplace=True)
df1['year'] = df1['year'].astype('int')
df1['landcover'] = df1['landcover'].map(landcover_types)

In [8]:
print(len(df1))
print(df1.dtypes)
df1.sample(5)

9856
values       float64
landcover     object
geo           object
year           int64
dtype: object


,values,landcover,geo,year
7830,3.1,Water,EU27_2020,2015
7116,6.7,Bare land,PT15,2009
5525,NaN,Grassland,FRG0,2012
9553,NaN,Wetland,PL72,2012
4659,19.7,Shrubland,PT16,2018


In [9]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
dir = os.path.join('gdrive', 'MyDrive', 'EuroStat', '01 - Intro to Python for Data Science')
data_dir = os.path.join(dir, 'data')
os.makedirs(data_dir, exist_ok=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Folium
https://pypi.org/project/folium/

In [10]:
m_1 = folium.Map(location=[49.63321762577624, 6.169436798146017], tiles='openstreetmap', zoom_start=17)
m_1

In [11]:
help(folium.Map)

Help on class Map in module folium.folium:

class Map(branca.element.MacroElement)
 |  Map(location=None, width='100%', height='100%', left='0%', top='0%', position='relative', tiles='OpenStreetMap', API_key=None, max_zoom=18, min_zoom=0, max_native_zoom=None, zoom_start=10, world_copy_jump=False, no_wrap=False, attr=None, min_lat=-90, max_lat=90, min_lon=-180, max_lon=180, max_bounds=False, detect_retina=False, crs='EPSG3857', control_scale=False, prefer_canvas=False, no_touch=False, disable_3d=False, subdomains='abc', png_enabled=False, zoom_control=True)
 |  
 |  Create a Map with Folium and Leaflet.js
 |  
 |  Generate a base map of given width and height with either default
 |  tilesets or a custom tileset URL. The following tilesets are built-in
 |  to Folium. Pass any of the following to the "tiles" keyword:
 |  
 |      - "OpenStreetMap"
 |      - "Mapbox Bright" (Limited levels of zoom for free tiles)
 |      - "Mapbox Control Room" (Limited levels of zoom for free tiles)
 |  

In [12]:
tiles = [name.strip() for name in """
    OpenStreetMap
    Stamen Terrain
    Stamen Toner
    Stamen Watercolor
    CartoDB positron
    CartoDB dark_matter""".strip().split('\n')]

@interact(tiles=tiles)
def create_map(tiles="Open Streetmap"):
    return folium.Map(location=[49.63321762577624, 6.169436798146017], tiles=tiles, zoom_start=15, width='90%')

interactive(children=(Dropdown(description='tiles', options=('OpenStreetMap', 'Stamen Terrain', 'Stamen Toner'…

#Choropleth maps

## NUTS2 polygons
Nomenclature of Territorial Units for Statistics

https://ec.europa.eu/eurostat/web/nuts/background

https://ec.europa.eu/eurostat/web/gisco/geodata/reference-data/administrative-units-statistical-units/nuts

In [13]:
%%html
<iframe src="https://epsg.io/4326" width="1000" height="600"></iframe>

In [14]:
%%bash
mkdir -p nuts
curl https://gisco-services.ec.europa.eu/distribution/v2/nuts/download/ref-nuts-2021-10m.shp.zip --output nuts/ref-nuts-2021-10m.shp.zip
cd nuts
unzip ref-nuts-2021-10m.shp.zip
unzip NUTS_RG_10M_2021_4326_LEVL_2.shp.zip -d NUTS_RG_10M_2021_4326_LEVL_2.shp
mv ./NUTS_RG_10M_2021_4326_LEVL_2.shp '../gdrive/MyDrive/EuroStat/01 - Intro to Python for Data Science/data'

Archive:  ref-nuts-2021-10m.shp.zip
  inflating: NUTS_RG_10M_2021_3035.shp.zip  
  inflating: NUTS_RG_10M_2021_3035_LEVL_0.shp.zip  
  inflating: NUTS_RG_10M_2021_3035_LEVL_1.shp.zip  
  inflating: NUTS_RG_10M_2021_3035_LEVL_2.shp.zip  
  inflating: NUTS_RG_10M_2021_3035_LEVL_3.shp.zip  
  inflating: NUTS_RG_10M_2021_3857.shp.zip  
  inflating: NUTS_RG_10M_2021_3857_LEVL_0.shp.zip  
  inflating: NUTS_RG_10M_2021_3857_LEVL_1.shp.zip  
  inflating: NUTS_RG_10M_2021_3857_LEVL_2.shp.zip  
  inflating: NUTS_RG_10M_2021_3857_LEVL_3.shp.zip  
  inflating: NUTS_RG_10M_2021_4326.shp.zip  
  inflating: NUTS_RG_10M_2021_4326_LEVL_0.shp.zip  
  inflating: NUTS_RG_10M_2021_4326_LEVL_1.shp.zip  
  inflating: NUTS_RG_10M_2021_4326_LEVL_2.shp.zip  
  inflating: NUTS_RG_10M_2021_4326_LEVL_3.shp.zip  
  inflating: NUTS_LB_2021_3035.shp.zip  
  inflating: NUTS_LB_2021_3035_LEVL_0.shp.zip  
  inflating: NUTS_LB_2021_3035_LEVL_1.shp.zip  
  inflating: NUTS_LB_2021_3035_LEVL_2.shp.zip  
  inflating: NUTS_LB

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.1M  100 10.1M    0     0  6727k      0  0:00:01  0:00:01 --:--:-- 6727k
mv: inter-device move failed: './NUTS_RG_10M_2021_4326_LEVL_2.shp' to '../gdrive/MyDrive/EuroStat/01 - Intro to Python for Data Science/data/NUTS_RG_10M_2021_4326_LEVL_2.shp'; unable to remove target: Directory not empty


In [15]:
%%bash
cd nuts
rm -r *

In [16]:
nuts2 = gpd.read_file(os.path.join(data_dir, 'NUTS_RG_10M_2021_4326_LEVL_2.shp'))
nuts2.set_index('NUTS_ID', inplace=True)

In [17]:
nuts2.sample(n=5)

,LEVL_CODE,CNTR_CODE,NAME_LATN,NUTS_NAME,MOUNT_TYPE,URBN_TYPE,COAST_TYPE,FID,geometry
NUTS_ID,,,,,,,,,
SE21,2,SE,Småland med öarna,Småland med öarna,0.0,NaN,0,SE21,"MULTIPOLYGON (((19.16235 57.99255, 19.24167 57..."
HU31,2,HU,Észak-Magyarország,Észak-Magyarország,0.0,NaN,0,HU31,"POLYGON ((22.12108 48.37831, 22.07122 48.30792..."
TR21,2,TR,"Tekirdağ, Edirne, Kırklareli","Tekirdağ, Edirne, Kırklareli",0.0,NaN,0,TR21,"POLYGON ((28.03551 41.98308, 28.04658 41.93799..."
TRC1,2,TR,"Gaziantep, Adıyaman, Kilis","Gaziantep, Adıyaman, Kilis",0.0,NaN,0,TRC1,"POLYGON ((39.12275 38.18331, 39.22742 38.19483..."
RO12,2,RO,Centru,Centru,0.0,NaN,0,RO12,"POLYGON ((25.66158 47.09165, 25.73840 47.06297..."


## Choropleth

In [18]:
landcover = 'Woodland'
year = 2018
data = df1[(df1['landcover']==landcover) & (df1['year']==year)].set_index('geo')['values']
data

geo
AT11    31.9
AT12    39.1
AT13    28.5
AT21    57.8
AT22    55.5
        ... 
UKM6    13.1
UKM7    14.7
UKM8    19.3
UKM9    22.5
UKN0     8.1
Name: values, Length: 308, dtype: float64

In [19]:
m_2 = folium.Map(location=[49.63321762577624, 6.169436798146017],
                 tiles='CartoDB positron',
                 zoom_start=4)

Choropleth(geo_data=nuts2.__geo_interface__, 
           key_on="feature.id", 
           data=data,
           fill_color='YlGnBu', 
           legend_name=f'Percentage land coved by {landcover} in {year}'
          ).add_to(m_2)

m_2

### ❓ Exercise

In [20]:
landcover = random.choice(list(landcover_types.values()))
year = random.choice(df1.year.unique())
md(f"##❓ Plot the percentage of land covered by {landcover} in {year} at country-level!")

##❓ Plot the percentage of land covered by Water in 2018 at country-level!

# Plotting Points

### LAU data

Local Administrative Units

https://ec.europa.eu/eurostat/web/nuts/local-administrative-units

In [21]:
df3 = pd.read_excel(os.path.join(data_dir, 'EU-28-LAU-2019-NUTS-2016.xlsx'),
                    sheet_name='LU')

In [22]:
df3.sample(n=10)

,NUTS 3 CODE,LAU CODE,LAU NAME NATIONAL,LAU NAME LATIN,CHANGE (Y/N),POPULATION,TOTAL AREA (m2),DEGURBA,DEG change compared to last year,COASTAL AREA (yes/no),COAST change compared to last year,CITY_ID,CITY_ID change compared to last year,CITY_NAME,GREATER_CITY_ID,GREATER_CITY_ID change compared to last year,GREATER_CITY_NAME,FUA_ID,FUA_ID change compared to last year,FUA_NAME
58,LU000,610,Vallée de l'Ernz,Vallée de l'Ernz,no,2641,39730000.0,3,no,no,NaN,NaN,no,NaN,NaN,no,NaN,LU001L1,no,Luxembourg
3,LU000,104,Käerjeng,Käerjeng,no,10409,33670000.0,2,no,no,NaN,NaN,no,NaN,NaN,no,NaN,LU001L1,no,Luxembourg
20,LU000,212,Rumelange,Rumelange,no,5608,6830000.0,2,no,no,NaN,NaN,no,NaN,NaN,no,NaN,LU001L1,no,Luxembourg
75,LU000,808,Winseler,Winseler,no,1365,30420000.0,3,no,no,NaN,NaN,no,NaN,NaN,no,NaN,LU001L1,no,Luxembourg
94,LU000,1201,Bous,Bous,no,1652,15430000.0,3,no,no,NaN,NaN,no,NaN,NaN,no,NaN,LU001L1,no,Luxembourg
10,LU000,202,Differdange,Differdange,no,26796,22180000.0,2,no,no,NaN,NaN,no,NaN,NaN,no,NaN,LU001L1,no,Luxembourg
65,LU000,707,Saeul,Saeul,no,844,14860000.0,3,no,no,NaN,NaN,no,NaN,NaN,no,NaN,LU001L1,no,Luxembourg
56,LU000,608,Reisdorf,Reisdorf,no,1234,14840000.0,3,no,no,NaN,NaN,no,NaN,NaN,no,NaN,LU001L1,no,Luxembourg
23,LU000,301,Bertrange,Bertrange,no,8282,17390000.0,2,no,no,NaN,NaN,no,NaN,NaN,no,NaN,LU001L1,no,Luxembourg
45,LU000,502,Wincrange,Wincrange,no,4392,113360000.0,3,no,no,NaN,NaN,no,NaN,NaN,no,NaN,LU001L1,no,Luxembourg


## Geocoding

In [23]:
%%time
result = geocode(df3['LAU NAME LATIN'].map(lambda name: name + " Luxembourg"),
                 provider="nominatim")
result

/usr/local/lib/python3.7/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


CPU times: user 584 ms, sys: 87 ms, total: 671 ms
Wall time: 50.7 s


In [24]:
df4 = pd.concat([df3[['LAU NAME LATIN', 'POPULATION']], result], axis=1)
df4['Lat']= df4['geometry'].map(lambda p: p.y)
df4['Long'] = df4['geometry'].map(lambda p: p.x)
df4.sample(n=10)

,LAU NAME LATIN,POPULATION,geometry,address,Lat,Long
29,Schuttrange,4166,POINT (6.27143 49.62267),"Schuttrange, Canton Luxembourg, Lëtzebuerg",49.622668,6.271431
33,Weiler-la-Tour,2400,POINT (6.19956 49.54170),"Weiler-la-Tour, Canton Luxembourg, Lëtzebuerg",49.541697,6.199557
38,Heffingen,1468,POINT (6.24052 49.77079),"Heffingen, Canton Mersch, Lëtzebuerg",49.770788,6.240520
78,Vianden,2080,POINT (6.20667 49.93370),"Vianden, Canton Vianden, Lëtzebuerg",49.933698,6.206671
31,Strassen,9589,POINT (6.05838 49.63192),"Strassen, Canton Luxembourg, Lëtzebuerg",49.631923,6.058375
56,Reisdorf,1234,POINT (6.27087 49.87268),"Reisdorf, Canton Diekirch, Lëtzebuerg",49.872683,6.270869
0,Dippach,4333,POINT (5.98148 49.58751),"Dippach, Canton Capellen, Lëtzebuerg",49.587507,5.981479
80,Bech,1259,POINT (6.36266 49.75201),"Bech, Canton Echternach, Lëtzebuerg",49.752011,6.362657
66,Useldange,1900,POINT (5.98154 49.76967),"Useldange, Canton Redange, 8706, Lëtzebuerg",49.769668,5.981543
76,Tandel,2083,POINT (6.17473 49.90816),"Tandel, Canton Vianden, Lëtzebuerg",49.908162,6.174731


## Marker

In [25]:
m_4 = folium.Map(location=[49.73321762577624, 6.169436798146017],
                 tiles='CartoDB positron',
                 zoom_start=9)

for idx, row in df4.iterrows():
    Marker([row['Lat'], row['Long']], popup=f"{row['LAU NAME LATIN']}\n{row['POPULATION']} inhabitants").add_to(m_4)

m_4

## Markercluster

In [26]:
m_5 = folium.Map(location=[49.73321762577624, 6.169436798146017],
                 tiles='CartoDB positron',
                 zoom_start=9)

mc = MarkerCluster()
for idx, row in df4.iterrows():
    mc.add_child(Marker([row['Lat'], row['Long']], 
                        popup=f"{row['LAU NAME LATIN']}\n{row['POPULATION']} inhabitants"))
m_5.add_child(mc)

m_5

## Bubbles

In [27]:
m_6 = folium.Map(location=[49.73321762577624, 6.169436798146017],
                 tiles='CartoDB positron',
                 zoom_start=9)

for idx, row in df4.iterrows():
  Circle(
    location=[row['Lat'], row['Long']],
    radius=100 * np.log(row['POPULATION']),
    color='firebrick' if row['POPULATION']>=5000 else 'peru').add_to(m_6)

m_6

## Heatmap

In [28]:
m_7 = folium.Map(location=[49.73321762577624, 6.169436798146017],
                 tiles='CartoDB positron',
                 zoom_start=9)

HeatMap(data=[[row['Lat'], row['Long'], row['POPULATION']] for _, row in df4.iterrows()],
        radius=25,
        blur=15,
        max_val = max(df4['POPULATION']) / 1000).add_to(m_7)

m_7

In [29]:
help(HeatMap)

Help on class HeatMap in module folium.plugins.heat_map:

class HeatMap(folium.map.Layer)
 |  HeatMap(data, name=None, min_opacity=0.5, max_zoom=18, max_val=1.0, radius=25, blur=15, gradient=None, overlay=True, control=True, show=True)
 |  
 |  Create a Heatmap layer
 |  
 |  Parameters
 |  ----------
 |  data : list of points of the form [lat, lng] or [lat, lng, weight]
 |      The points you want to plot.
 |      You can also provide a numpy.array of shape (n,2) or (n,3).
 |  name : string, default None
 |      The name of the Layer, as it will appear in LayerControls.
 |  min_opacity  : default 1.
 |      The minimum opacity the heat will start at.
 |  max_zoom : default 18
 |      Zoom level where the points reach maximum intensity (as intensity
 |      scales with zoom), equals maxZoom of the map by default
 |  max_val : float, default 1.
 |      Maximum point intensity
 |  radius : int, default 25
 |      Radius of each "point" of the heatmap
 |  blur : int, default 15
 |      Am